# Moment Comparison Between Microscopy & Flow Cytometry

In [1]:
import numpy as np
import pandas as pd
import glob
import mwc_induction_utils as mwc
import scipy.stats
import pymc3 as pm 
import theano.tensor as tt
import corner
from tqdm import tqdm
import scipy.optimize

import seaborn as sns
import matplotlib.pyplot as plt
import bokeh.io
import bokeh.plotting
import bokeh.layouts
np.random.seed(42)
bokeh.io.output_notebook()

Loading BokehJS ...

##  Comparison of distributions

In [27]:
# Load the microscopy data set.
mic_files = glob.glob('../data/microscopy/*O2*microscopy*.csv')
mic_df = pd.concat([pd.read_csv(f, comment='#') for f in mic_files])

# Only keep RBS1027 for the purposes of this comparison.
mic_df = mic_df[mic_df['rbs']=='RBS1027']

# Load the flow data sets.
flow_files = glob.glob('../data/flow/*O2*IPTG.csv')
flow_df = []
alpha = 0.4
for i, file in enumerate(flow_files):
    # Load the data set and gate
    _df = pd.read_csv(file, comment='#')
    gated = mwc.auto_gauss_gate(_df, alpha=alpha)
    
    # Get the string of the concentration. 
    name_split = file.split('_')
    conc = float(name_split[-1].split('uM')[0])
    date = name_split[0].split('/')[-1]
    operator = name_split[2]
    rbs = name_split[3]
    _df = _df.loc[:, ['FITC-A', 'FSC-A', 'SSC-A']]
    _df.insert(0, 'rbs', rbs)
    _df.insert(0, 'operator', operator)
    _df.insert(0, 'IPTG_uM', conc)
    _df.insert(0, 'date', date)
    flow_df.append(_df)
    
flow_df = pd.concat(flow_df, axis=0)


In [28]:
def ecdf(data):
    return np.sort(data), np.arange(0, len(data)) / len(data)

In [4]:
plot_kwargs = {'plot_width':500, 'plot_height':300}
p1 = mwc.bokeh_boiler(**plot_kwargs)
p2 = mwc.bokeh_boiler(**plot_kwargs)
grouped = mic_df.groupby('IPTG_uM')
colors = sns.color_palette('Blues_r', n_colors=20)
colors = colors.as_hex()
i = 0 
for g, d_mic in grouped: 
    x, y = ecdf(d_mic['mean_intensity'])
    p1.line(np.log10(x), y, line_width=2.5, color=colors[i],
            alpha=0.75) 
    i += 1  
    
grouped = flow_df.groupby('IPTG_uM')
i = 0
for g, d_flow in grouped:
    x, y = ecdf(d_flow['FITC-A'])
    p2.line(np.log10(x), y, line_width=2.5, color=colors[i])
    i += 1
p1.xaxis.axis_label = 'log mean pixel intensity (a.u.)'
p1.yaxis.axis_label = 'ECDF'
p1.title.text = 'distributions from microscopy'

p2.xaxis.axis_label='log mean object intensity'
p2.title.text = 'distributions from flow cytometry'

row = bokeh.layouts.row(p1, p2)
# bokeh.io.show(row)
                       

To compare the distributions between the two methods, we can look at the **[central moments]()**, which provides information on the shape of the distribution rather than just the location. We can mathematically describe the $k-$th central moment of a data set as 

$$
m_k = {1 \over n}\sum\limits_i^n(x_i - \bar{x})^k,
$$

where $m_k$ is the $k-$th moment, $x_i$ is the $i$-th data point in the data set, and $\bar{x}$ is the mean.


In [30]:
p = mwc.bokeh_boiler(**{'plot_width': 500, 'plot_height': 300, 'y_axis_type':'log'})

moments = [2, 3, 4, 5, 6, 7]

flow_dup_files = glob.glob('../data/5000uM_IPTG_distributions/*IPTG.csv')
for f in flow_dup_files:
    d_flow = pd.read_csv(f, comment='#')
    gated = mwc.auto_gauss_gate(d_flow, alpha=0.4)
    for m in moments:
#     _mic_moment = scipy.stats.moment(d_mic['mean_intensity'], moment=m) 
        _flow_moment = scipy.stats.moment(gated['FITC-A'], moment=m) 
#     p.circle(m, _mic_moment, size=10, legend='microscopy', color='dodgerblue')
        p.circle(m, _flow_moment, size=5, legend='flow cytometry', color='tomato',
                alpha=0.5)
    
mic_dup_files = glob.glob('../data/5000uM_IPTG_distributions/*microscopy*.csv')
for f in mic_dup_files:
    d_mic = pd.read_csv(f, comment='#')
    d_mic = d_mic[d_mic['IPTG_uM']==5000]
    for m in moments:
        _mic_moment = scipy.stats.moment(d_mic['mean_intensity'], moment=m)
        p.circle(m, _mic_moment, size=5, legend='microscopy', color='dodgerblue',
                alpha=0.5)
    
p.xaxis.axis_label = 'central moment order'
p.yaxis.axis_label = 'log central moment value'
p.legend.location = 'top_left'
p.legend.background_fill_color = '#E3DCD0'
p.legend.border_line_color = '#FFFFFF'
bokeh.io.show(p)
    

## Parameter estimation using population means 

In [31]:
# Randomly choose three days. 
flow_master = pd.read_csv('../data/flow_master.csv', comment='#')
O2_flow = flow_master[((flow_master['operator']=='O2') |
                      (flow_master['operator']=='O1')) &
                      (flow_master['rbs']=='RBS1027')]
# dates = O2_flow['date'].unique()
# chosen_dates = np.random.choice(dates, 3)                   
# O2_flow = O2_flow[(O2_flow['date']== chosen_dates[0]) |
#                  (O2_flow['date']== chosen_dates[1]) |
#                  (O2_flow['date']== chosen_dates[2])]

p = mwc.bokeh_boiler(**{'plot_width': 500, 'plot_height':300,
                        'x_axis_type':'log', 'x_axis_label':'[IPTG] (µM)',
                       'y_axis_label':'fold-change'})

grouped = O2_flow.groupby('IPTG_uM')['fold_change_A'].mean()
IPTG = O2_flow['IPTG_uM'].unique() / 1E6
p.circle(O2_flow['IPTG_uM'] / 1E6, O2_flow['fold_change_A'], size=5,
        color='dodgerblue')
# p.circle(IPTG, grouped, size=7, color='dodgerblue')


# Microscopy. 
mic_master = pd.read_csv('../data/microscopy_master.csv', comment='#')
O2_mic = mic_master[((mic_master['operator']=='O2') | (mic_master['operator'] == 'O1')) &\
                    (mic_master['rbs']=='RBS1027')]
grouped = O2_mic.groupby('IPTG_uM')['fold_change'].mean()
# p.circle(IPTG, grouped, color='tomato')
p.circle(O2_mic['IPTG_uM']/1E6, O2_mic['fold_change'], color='tomato')

bokeh.io.show(p)

In [7]:
# Compare the hpd of the values. 
R = 260  # Number of repressor molecules
ep_r = -13.9 #Binding energy of repressor to the DNA
ep_ai = 4.5  # Energetic difference between allosteric states
fixed_params = [R, ep_r, ep_ai]

# Set up the model and define the priors. 
def jeffreys(val):
    return -tt.log(val)

def traceframe(trace, model):
    df = pm.trace_to_dataframe(trace)
    lnprob = []
    for t in tqdm(trace):
        lnprob.append(model.logp(t))
    df.insert(np.shape(df)[1], 'lnprob', lnprob)        
    return df



In [8]:
flow_model = pm.Model()
mic_model = pm.Model()
models = [flow_model, mic_model]
methods = ['flow cytometry', 'microscopy']
model_df = [O2_flow, O2_mic]
N_ns = 4.6E6

ep_r = -13.9
dfs = []
for i, m in enumerate(models):
    IPTG = model_df[i]['IPTG_uM'].values
    if i == 0:
        fc_obs = model_df[i]['fold_change_A'].values
    else:
        fc_obs = model_df[i]['fold_change'].values
    
    with m:
        ep_a = pm.Uniform('ep_a', lower=-10, upper=10, testval=5)
        ep_i = pm.Uniform('ep_i', lower=-10, upper=10, testval=-0.5)
        sigma = pm.HalfNormal('sigma', sd=1)
#         sigma = pm.DensityDist('sigma', jeffreys, testval=1)

        p_act = (1 + IPTG * np.exp(-ep_a))**2 / ((1 + IPTG * np.exp(-ep_a))**2 +\
            np.exp(-ep_ai) * (1 + IPTG * np.exp(-ep_i))**2)
        fc_theo = (1 + p_act * (2 * model_df[i]['repressors'] / N_ns) * np.exp(-model_df[i]['binding_energy']))**-1
        like = pm.Normal('likelihood',mu=fc_theo, sd=np.sqrt(sigma), observed=fc_obs)
        
        # Find the map. 
        _MAP = pm.find_MAP(model=m, fmin=scipy.optimize.fmin_powell)
        trace = pm.sample(start=_MAP, draws=10000, tune=10000)
        
    df = traceframe(trace, model=m)
#     df.insert(0, 'method', methods[i])
    dfs.append(df)
 

Auto-assigning NUTS sampler...
Initializing NUTS using advi...


Optimization terminated successfully.
         Current function value: -380.438966
         Iterations: 3
         Function evaluations: 124


Average ELBO = 369.29: 100%|██████████| 200000/200000 [00:18<00:00, 11021.77it/s]
Finished [100%]: Average ELBO = 369.29
100%|██████████| 10000/10000 [00:00<00:00, 11210.42it/s]
Auto-assigning NUTS sampler...
Initializing NUTS using advi...


Optimization terminated successfully.
         Current function value: -68.357939
         Iterations: 4
         Function evaluations: 171


Average ELBO = 60.011: 100%|██████████| 200000/200000 [00:17<00:00, 11748.44it/s]
Finished [100%]: Average ELBO = 60.023
100%|██████████| 10000/10000 [00:00<00:00, 11255.79it/s]


In [9]:
p = mwc.bokeh_boiler()

colors = ['dodgerblue', 'tomato']
for i, d in enumerate(dfs):
    x, y = ecdf(d['ep_a'])
    p.line(x,y, color=colors[i], line_width=2)

bokeh.io.show(p)

In [10]:
bin_min = np.min([np.min(dfs[0].ep_i), np.min(dfs[1].ep_i)])
bin_max = np.max([np.max(dfs[0].ep_i), np.max(dfs[1].ep_i)])
bins = np.linspace(np.exp(bin_min), np.exp(bin_max), 65)
flow_y, flow_x = np.histogram(np.exp(dfs[0].ep_i), bins=bins)
mic_y, mic_x = np.histogram(np.exp(dfs[1].ep_i), bins=bins)

p1 = mwc.bokeh_boiler()
p2 = mwc.bokeh_boiler()
p3 = mwc.bokeh_boiler()


colors = ['slategray', 'skyblue']
methods = ['flow cytometry', 'microscopy']
for i, d in enumerate(dfs):
    # Compute the ecdf. 
    ka_x, ka_y = ecdf(np.exp(d['ep_a']))
    ki_x, ki_y = ecdf(np.exp(d['ep_i']))
    p2.line(ka_x, ka_y, line_width=2, color=colors[i]) 
    p3.line(ki_x, ki_y, line_width=2, color=colors[i])
 


p1.quad(top=mic_y/np.sum(mic_y), bottom=0, left=mic_x[:-1], right=mic_x[1:], color='skyblue',
      alpha=0.8)
p1.quad(top=flow_y/np.sum(flow_y), bottom=0, left=flow_x[:-1], right=flow_x[1:], color='slategray',
      alpha=0.5)

ell = bokeh.layouts.layout([[p1, p2], [p3]], sizing_mode='stretch_both')
# bokeh.io.show(ell)
# bokeh.io.export_png(ell, filename='test.png')
# bokeh.io.show(p2)



In [11]:
bin_min = np.min([np.min(dfs[0].ep_i), np.min(dfs[1].ep_i)])
bin_max = np.max([np.max(dfs[0].ep_i), np.max(dfs[1].ep_i)])
bins = np.linspace(np.exp(bin_min), np.exp(bin_max), 65)
flow_y, flow_x = np.histogram(np.exp(dfs[0].ep_i), bins=bins)
mic_y, mic_x = np.histogram(np.exp(dfs[1].ep_i), bins=bins)

p1 = mwc.bokeh_boiler()
p2 = mwc.bokeh_boiler()
p3 = mwc.bokeh_boiler()


colors = ['slategray', 'skyblue']
methods = ['flow cytometry', 'microscopy']
for i, d in enumerate(dfs):
    # Compute the ecdf. 
    ka_x, ka_y = ecdf(np.exp(d['ep_a']))
    ki_x, ki_y = ecdf(np.exp(d['ep_i']))
    p2.line(ka_x, ka_y, line_width=2, color=colors[i]) 
    p3.line(ki_x, ki_y, line_width=2, color=colors[i])
 


p1.quad(top=mic_y/np.sum(mic_y), bottom=0, left=mic_x[:-1], right=mic_x[1:], color='skyblue',
      alpha=0.8)
p1.quad(top=flow_y/np.sum(flow_y), bottom=0, left=flow_x[:-1], right=flow_x[1:], color='slategray',
      alpha=0.5)

ell = bokeh.layouts.layout([[p1, p2], [p3]], sizing_mode='stretch_both')
# bokeh.io.show(ell)
# bokeh.io.export_png(ell, filename='test.png')
bokeh.io.show(p2)

